# Atomic resolution STEM data analysis

In this part of the tutorial, we will apply a variety of atomic resolution image processing techniques to analyze a STEM image of a perovskite Nd$_{0.5}$Sr$_{0.5}$MnO$_3$ thin film. We will use the `kemstem` package ([github](https://github.com/noahschnitzer/kemstem)), which can be installed on your own computer with `pip install kemstem`. 

In particular, we will:
* Fourier filter the image to identify structural domains
* Measure the strain field across the image
* Identify the A-site atomic columns in the image and fit them
* Measure periodic lattice displacements driven by charge ordering in the film with a specialized Fourier analysis technique

Alternative tools for atomic resolution image processing which may also be useful but won't be introduced here include:
* [atomap](https://atomap.org/)
* [stemtool](https://github.com/stemtool/stemtool)

The data used for this tutorial is from the publication [El Baggari, I., Baek, D. J., Zachman, M. J., Lu, D., Hikita, Y., Hwang, H. Y., ... & Kourkoutis, L. F. (2021). Charge order textures induced by non-linear couplings in a half-doped manganite. Nature communications, 12(1), 3747.](https://doi.org/10.1038/s41467-021-24026-7)

## [Download the data through PARADIM](https://data.paradim.org/doi/bg5n-4s68/Figure%20S3%20Twins/30_cryo_0716_NSMO110_2-55Mx_0p6us_1024px_REGISTERED.tif)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tifffile import imread
import kemstem
%matplotlib widget

# Data loading and preprocessing
We begin by loading an atomic resolution HAADF STEM image of our film, and performing some basic preprocessing before beginning analysis. Here, the image is cropped to a square for ease of analysis and the intensity is normalized between 0 and 1 for interpretability.

In [ ]:
filename = '/Users/noahschnitzer/Downloads/30_cryo_0716_NSMO110_2-55Mx_0p6us_1024px_REGISTERED.tif'
image = imread(filename)
print(f'Image shape {image.shape}')
image = image[:min(image.shape),:min(image.shape)]
print(f'Cropped to {image.shape}')
image = image - image.min()
image = image / image.max()
print(f'Normalized to min: {image.min()}, max: {image.max()}')

fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(8,8))
ax.matshow(image,cmap='gray')

In the above image of our perovskite film, the A-site columns (a random mixture of Nd and Sr) show up as bright white blobs, while the B-site columns (Mn) are less intense. The oxygen in the structure is not visible. If you look very closely you may be able to just make out two distinct domains.

We'll next take a Fourier Transform of the image to get an idea of the reciprocal space structure of our film.

In [ ]:
pattern_c = kemstem.fourier.prepare_fourier_pattern(image)
pattern_log = kemstem.fourier.prepare_fourier_pattern(image,log=True) # Here we also take a log transform of the FFT for ease of visualization

fig,ax = plt.subplots(1,2,constrained_layout=True,figsize=(10,5))
ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(pattern_log,cmap='gray')
ax[1].axis('off')

In the above FFT, bright Bragg peaks are visible as well as less intense additional reflections. These less intense reflections are satellite peaks describing a superlattice. For now, we'll focus on using Fourier analysis to isolate individual peaks and analyze domains and strain.

# Fourier analysis: filtering and phase lock-in
We'll begin by selecting some of the peaks. To do so, simply run the following cell and click on some of the peaks visible in the Fourier pattern. Be sure to include the intense peak around (495, 380) in your selection, as well as some of the other Bragg peaks and superlattice peaks

In [ ]:
selected_peaks = kemstem.fourier.select_peaks(pattern_log,zoom=300,select_conjugates=False,figsize=(5,5),delete_within=3)

Once you've selected your peaks of interest, run the below cell to refine the peak positions. Here we convolve the pattern with a Gaussian and then fit a Gaussian to each peak to find its center. For some of the following analysis, it is important that the peak positions are identified as accurately as possible. In the following plot the hand-picked positions are marked in blue and the refined positions in red.

In [ ]:
p0 = np.array(selected_peaks).T
print(f'Selected X positions: {p0[:,1]}')
print(f'Selected Y positions: {p0[:,0]}')

peaks_ref = kemstem.fourier.refine_peaks_gf(gaussian_filter(pattern_log,.5), p0, window_dimension=5,store_fits=False, remove_unfit = False)[0]
fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(5,5))
kemstem.util.plot_numbered_points(pattern_log,p0,ax=ax,color='b',zoom=200)
ax.plot(peaks_ref[:,1],peaks_ref[:,0],'r.')
ax.axis('off')

Next, we'll individually fourier filter each of our peaks. Fourier filtering means we're selecting a single peak of interest and mapping out the information from only that spacing. This can be very useful for understanding where different features in your image or fourier transforms are coming from.

Practically, we do this by applying a Gaussian mask around the peak. The width (standard deviation) of this Gaussian, which is set by the `sigma` variable in the cell below, is a very important parameter. Choosing a larger `sigma` will incorporate more noise into the filter, and if it is too large information from other peaks may even leak in. On the other hand, choosing too small of a `sigma` will limit the real space resolution of the fourier filtered map. In the below cell, try filtering different peaks selected above by changing the `peak_index` variable, and also try to find a reasonable value of `sigma`.
The following figure will plot:
1) On the left, the absolute value of the fourier filtered signal, which shows how the intensity of the filtered peak varies in real space.
2) In the middle, the real part of the fourier filtered signal, which produces a grating corresponding to the filtered peak. This grating incorporates phase information, visible as bending or discontinuities in the stripes, which can offer useful information about defects and other structural changes.
3) On the right, the fourier transform is shown with Gaussian used to generate the Fourier filter overlaid. This shows which peak is being filtered, as well as what reciprocal space information is being incorporated.

In [ ]:
peak_index = 1
sigma= 5
filtered_im, filtered_ft, mask = kemstem.fourier.fourier_filter(pattern_c,peaks_ref[peak_index,:],sigma=sigma)

fig,ax = plt.subplots(1,3,constrained_layout=True,figsize=(12,4))
ax[0].matshow(image,cmap='gray')
ax[0].matshow(np.abs(filtered_im),cmap='inferno',alpha=.5)
ax[0].axis('off')
ax[1].matshow(np.real(filtered_im),cmap='gray')
ax[1].axis('off')
ax[2].matshow(pattern_log,cmap='gray')
ax[2].matshow(mask,alpha=.5,cmap='inferno')
ax[2].axis('off')

Before proceeding, set the peak index above so that the Bragg peak indicated in this image is selected:


Next, we'll extract the phase information associated with this peak from the image. The phase of a peak offers rich information about strain and defects associated with that orientation / spacing. The approach used here is a Phase lock-in technique, as described in [Goodge, B. H., El Baggari, I., Hong, S. S., Wang, Z., Schlom, D. G., Hwang, H. Y., & Kourkoutis, L. F. (2022). Disentangling coexisting structural order through phase lock-in analysis of atomic-resolution STEM data. Microscopy and Microanalysis, 28(2), 404-411.](https://doi.org/10.1017/S1431927622000125), but the result is comparable to Geometric Phase Analysis (GPA) which is also commonly used.

In [ ]:
phase = kemstem.fourier.phaselock(filtered_im,peaks_ref[peak_index,:],sigma=20) # different from the sigma above 
# this sigma is a parameter of the lock in analysis and should generally be larger than the sigma used for Fourier filtering
# but should not need to be changed here

fig,ax = plt.subplots(1,3,constrained_layout=True,figsize=(12,4))

ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(np.abs(filtered_im),cmap='inferno')
ax[1].axis('off')
kemstem.util.plot_phase(phase,ax=ax[2])
ax[2].axis('off')

Above are plotted:
1) On the left, the original image
2) In the middle, the amplitude of the fourier filtered peak
3) On the right, the phase of the fourier filtered peak, with contours drawn at each $\pi$/3 step. 

At a glance the phase doesn't seem to tell us much, but in the next cell we'll take the gradient of the phase to convert it into a strain field. In particular, we'll plot out the longitudinal and transverse components of the strain, giving us an idea of the local tensile and shear strains.

In [ ]:
# strain
sv = .02

ref_x = peaks_ref[peak_index,1]-image.shape[0]/2. 
ref_y = peaks_ref[peak_index,0]-image.shape[0]/2. 
eps_par, eps_trans = kemstem.fourier.phase_to_strain(phase, ref_x, ref_y, mask_threshold=1.)
fig,ax = plt.subplots(1,3,constrained_layout=True,figsize=(12,4),sharex=True,sharey=True)
ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(eps_par,cmap='bwr',vmin=-sv,vmax=sv)
ax[1].axis('off')
ax[2].matshow(eps_trans,cmap='bwr',vmin=-sv,vmax=sv)
ax[2].axis('off')


Above are plotted:
1) On the left, the original image.
2) In the middle, the longitudinal component of the strain.
3) On the right, the transverse component of the strain.

For our Bragg peak of interest, you should see that the longitudinal strain shows some minor variations but for the most part is small and rather unfirom, while the transverse strain shows a clear boundary. This actually corresponds to a crystalline twin domain in the film!

Next, go back through this section with some of the other Bragg and superlattice peaks -- how do they compare? Do you notice any artifacts that emerge?
You may have to adjust the `sv` variable in the cell above to change the saturation of the strain maps.

Next up, we'll move onto some atomistic analysis.

# Atomistic analysis: Finding and fitting columns

Now, we'll begin to analyze the image atomic column by atomic column, and to start we'll try to find the positions of all the bright A-site columns.

To accomplish this, we'll first bandpass filter our image. This is an important step because our image has a large low frequency intensity ramp. Here, we construct an ad-hoc filter by subtracting off a low pass filtered signal (effectively generating a high-pass filtered signal), and then applying an additional low pass filter.

The original (left) and filtered (right) images are plotted in the figure below. 

In [ ]:
sigma_low = 1
sigma_high = 20

im_bandpass = kemstem.util.general.normalize(gaussian_filter(image- gaussian_filter(image,sigma_high),sigma_low))
fig,ax = plt.subplots(1,2,figsize = (10,5),constrained_layout=True)
ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(im_bandpass,cmap='gray')
ax[1].axis('off')

Next, we'll use a blob finder to identify the column positions. The blob finder used here takes two arguments:
1) `distance`, which sets the real space distance between peaks
2) `threshold`, which is used for additional filtering and sets a column intensity.

Try adjusting these parameters below to select all the intense A sites. While you may be able to adjust them to get the B sites as well, they won't be important to our analysis here so getting a clean selection of just the A sites is preferred. It's also ok if some A-sites are missed as long as most of them are selected.

In [ ]:
distance = 8
threshold = .1
c0 = kemstem.atomic.find_columns(im_bandpass,distance=distance, threshold=threshold) # 8 .2

fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(5,5))
ax.matshow(im_bandpass,cmap='gray')
ax.axis('off')
ax.plot(c0[:,1],c0[:,0],'r.',markersize=1)

Now that we've roughly identified the positions of our A-site cations, we'll refine the positions by fitting a Gaussian to each column. In the below cell a test fit is run for a single column. Adjust: 
* `sigma`, a Gaussian filter applied to the image to aid fitting
* `test_it`, the site being tested on
* `window_dim`, the size of the patch around each site being fit -- usually the most important parameter

until you think you've identified good parameters to use to fit all the columns we identified.

The figure generated by this cell shows:
* The patch of image data being fit in the top left
* The fit result in the top right (should look as similar as possible to the top left)
* The full image in the bottom left, marked with the original and refined positions in blue and red.
* The fit residual, which should be as small and flat as possible.

In [ ]:
sigma = 1
test_it = 555
window_dim = 7
fitting_image = gaussian_filter(image,sigma)
cf,errs,opts,data_fits = kemstem.atomic.columnfind.refine_columns(fitting_image,c0[test_it,:],window_dim)

fig,ax = plt.subplots(2,2,constrained_layout=True)
ax[0,0].matshow(data_fits[:,:,0,0],cmap='gray')
ax[0,1].matshow(data_fits[:,:,0,1],cmap='gray')
ax[1,0].matshow(fitting_image,cmap='gray')
ax[1,0].plot(c0[test_it,1],c0[test_it,0],'b.')
ax[1,0].plot(cf[0,1],cf[0,0],'r.')
ax[1,1].matshow(data_fits[:,:,0,0]-data_fits[:,:,0,1],cmap='gray',vmin=-.1,vmax=.1)
_ = [tax.axis('off') for tax in ax.ravel()]

Once you've identified good parameters above, run the following cell to fit all the columns in the image. This may take a few moments.

In [ ]:
cf,errs,opts,data_fits = kemstem.atomic.columnfind.refine_columns(fitting_image,c0,window_dim)

Below are plotted the original blob finder positions (blue) and fit positions (red). As can be seen zooming in, Gaussian fitting each column is essential for precicely and accurately determining its position.

In [ ]:
fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(6,6))
ax.matshow(image,cmap='gray')
ax.plot(c0[:,1],c0[:,0],'b.')
ax.plot(cf[:,1],cf[:,0],'r.')
ax.axis('off')

Just from these peak positions, we can already do some quick preliminary analysis. The following cell plots a pair correlation function showing the neighborhood of points surrounding each site. Because we only identified the A-site positions this is not so interesting on its own, but we can use this to easily find the vectors between columns to aid in quantifying the atomic scale structure in real space.

In [ ]:
neighborhood = kemstem.atomic.neighborhood.get_neighborhood(cf, cf[:,:], k=20)
clusters = kemstem.atomic.neighborhood.cluster_neighbors(neighborhood,n_clusters=40)


ax = kemstem.util.viz.neighborhood_scatter_plot(neighborhood,clusters=clusters)

For ease of picking vectors of interest, the points above have been clustered, where the red numbers indicate the cluster index. Pick a cluster close to the plot origin, and set the `cluster_index` variable below to that number.

The cell below uses that cluster centroid as a guess vector to, for every column identified previously, find the corresponding neighbor and measure the distance between them. These distances and the vectors connecting the pairs of sites are plotted below, as well as a histogram of the inter-site distances.

In [ ]:
cluster_index = 4
vects = kemstem.atomic.neighborhood.get_vector_to_neighbors(cf,cf[:,:],clusters[cluster_index,:],threshold=1)
fig,ax = plt.subplots(1,2,constrained_layout=True,figsize=(12,6))
ax[0].matshow(image,cmap='gray')
kemstem.util.viz.plot_scalar_bonds(ax[0],np.linalg.norm(vects,axis=1),cf[:,:],cf[:,:]+vects,linewidth=2,cmap='plasma')
ax[0].axis('off')
_ = ax[1].hist(np.linalg.norm(vects,axis=1),bins=100)

While just measuring inter-column distances can give some useful structural information, what makes this image particularly  interesting is the presence of periodic lattice distortions associated with charge order. These picometer scale displacements are extremely tricky to measure reliably and require more advanced techniques for reliable quantification. In the next section, we'll combine real and reciprocal space information to isolate these distortions.

# Periodic lattice displacement mapping
Our basic approach will be to generate a reference image without the periodic lattice distortions by masking out all the superlattice peaks, then mapping out the displacements between our measured column positions and this undistorted reference.

To begin, run the cell below, which will bring up a peak picker like the one used at the beginning of the notebook. This time, select only the __less intense superlattice satellite peaks__, and try to select as many as possible. 


Note that now conjugate peaks (opposite peaks across the center of the pattern) are now automatically selected and mistaken selection can be removed by clicking on the peak again. The pattern is saturated to make the superlattice peaks more visible.

In [ ]:
selected_peaks_PLD = kemstem.fourier.peakpick.select_peaks(pattern_log,zoom=350,select_conjugates=True,figsize=(5,5),delete_within=5,vmax=6)

Next, like previously, we'll refine the peak positions. For this technique it turns out getting the exact center of each peak is less important, what's more essential is actually fitting the background level near each peak.

In [ ]:
p0PLD = np.array(selected_peaks_PLD).T
peaks_fit, perrors,popts,pdf = kemstem.atomic.columnfind.refine_columns(gaussian_filter(np.abs(pattern_c),2), p0PLD, window_dimension=9,store_fits=True, remove_unfit = False)
fig,ax = plt.subplots(1,1,constrained_layout=True)
kemstem.util.viz.plot_numbered_points(pattern_log,p0PLD,ax=ax,color='b',zoom=350)
ax.plot(peaks_fit[:,1],peaks_fit[:,0],'r.')
ax.axis('off')

The cell below will show a comparison of the peaks and the fits - it is not so important that the fits are perfect as that the background levels for each fit look approximately correct.

In [ ]:
_ = kemstem.util.viz.plot_fit_comparison(pdf,)

Next, we'll mask out each of the peaks by setting them to the background level determined by the fits. In this process only the amplitude is supressed - the phase information is retained - so side effects on the image should be minimal. Run the following cell and check in the resulting masked fourier transform that most / all the superlattice peaks are no longer visible. Plotted along side the masked fourier transform is the inverse transform - i.e. an image without the superlattice which we can use as an undistorted reference. This image should look almost exactly the same as the original image.

The mask size can be adjusted with the `mask_size` variable -- the mask should be large enough that the peaks are fully supressed, but small enough that no Bragg peaks are filtered.

In [ ]:
mask_size  = 9 
masked_pattern_c,masked_image = kemstem.fourier.fourierfilter.mask_peaks_circle(pattern_c,peaks_fit,popts[:,6],mask_size)
fig,ax = plt.subplots(1,2,constrained_layout=True,figsize=(10,5))
ax[0].matshow(np.log(np.abs(masked_pattern_c)),cmap='gray')
ax[0].axis('off')
ax[1].matshow(masked_image,cmap='gray')
ax[1].axis('off')


Next, to gain some intuition about how this process altered the image, we'll look at the difference between the original image and the masked image. The original image is plotted on the left, and is overlaid with the difference between the masked and original images on the right.

In [ ]:

difference_image = masked_image - image
difference_image = difference_image / np.abs(difference_image).max()
fig,ax = plt.subplots(1,2,constrained_layout=True,figsize=(10,5),sharex=True,sharey=True)
ax[0].matshow(image,cmap='gray')
ax[0].axis('off')
ax[1].matshow(image,cmap='gray')
ax[1].matshow(difference_image,cmap='bwr',alpha=.5,vmin=-.5,vmax=.5)
ax[1].axis('off')

Zooming into the difference image, a complex pattern should be visible. The key feature that we're most interested in are dumbell like features which should straddle most of the atomic columns -- these dumbells, with one side red and one side blue, indicate the column _shifted_ with the masking. These are the displacements that we are interested in mapping out.

Next, we'll fit all the A-sites in the masked reference image. For this the same blob finder identified sites and parameters determined previously can be used as the shifts are on the picometer scale - less than a pixel across in this image.

Do you notice any artifacts or interesting features in the difference image?

In [ ]:
c_masked = kemstem.atomic.columnfind.refine_columns(gaussian_filter(masked_image,1),c0,window_dim)[0]

Run the below cell to visualize the displacements. Do you notice any domains? How do they correspond with the twin domains identified previously?

In [ ]:
displacements = kemstem.atomic.periodicdistortion.measure_displacements(c_masked,cf,threshold=0.4)#kemstem.util.point.get_nearest_points(c_masked,cf,k=1)[0] - cf


scale = 2e3 # scale factor for the triangles
colors = 'angle' # 'angle' or 'mag'

fig,ax = plt.subplots(1,1,constrained_layout=True,figsize=(6,6))
ax.matshow(image,cmap='gray',alpha=.7)
kemstem.util.plot_displaced_site(cf,displacements,scale=scale,colors=colors,ax=ax,cmap='hsv',linewidth=.2,shape=5,angleshift=np.pi/2,scale_power=.5)#,disp_min=1e-2,disp_max = 1e-1,)
ax.axis('off')